In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./file/2.make_code.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Vehicle Type Code'] = input_['Vehicle Type Code'].astype(int)
input_['Make Code'] = input_['Make Code'].astype(int)
input_['Year Code'] = input_['Year Code'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            b = 0
            while True:
                b += 1

                try:
                    resp = requests.get(f'''https://www.moogparts.com/driv/partfinder/api.catalog.models?brand=moog&locale=en_US&country_code=US&year_id={input_.loc[a, 'Year Code']}&make_id={input_.loc[a, 'Make Code']}&vehicle_group_ids={input_.loc[a, 'Vehicle Type Code']}''',
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(10, 10))
                    
                    if resp.status_code == 200:
                        json_ = resp.json()

                        list_model_code = [dict_['id'] for dict_ in json_['models']]
                        if list_model_code:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame({'Vehicle Type Code': input_.loc[a, 'Vehicle Type Code'],
                                    'Make Code': input_.loc[a, 'Make Code'],
                                    'Model Code': list_model_code,
                                    'Year Code': input_.loc[a, 'Year Code']})
            
            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
            
            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'Vehicle Type Code']} > {input_.loc[a, 'Make Code']} > {input_.loc[a, 'Year Code']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Vehicle Type Code', 'Make Code', 'Model Code', 'Year Code'],
                                                ascending=[True, True, True, True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./file/3.model_code-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Vehicle Type Code', 'Make Code', 'Year Code'],
                                            ascending=[True, True, True],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/model_code_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：70] - 2 > 67 > 1978
[剩余数量：1000] - [当前时间：16:02:01]

[状态：ok，尝试次数：10] - 2 > 67 > 2009
[剩余数量：999] - [当前时间：16:02:02]

[状态：ok，尝试次数：66] - 2 > 67 > 1990
[剩余数量：998] - [当前时间：16:02:02]

[状态：ok，尝试次数：154] - 2 > 66 > 1977
[剩余数量：997] - [当前时间：16:02:02]

[状态：ok，尝试次数：60] - 2 > 67 > 1991
[剩余数量：996] - [当前时间：16:02:02]

[状态：ok，尝试次数：49] - 2 > 67 > 1981
[剩余数量：995] - [当前时间：16:02:02]

[状态：ok，尝试次数：47] - 2 > 67 > 1996
[剩余数量：994] - [当前时间：16:02:03]

[状态：ok，尝试次数：1] - 2 > 67 > 2019
[剩余数量：993] - [当前时间：16:02:03]

[状态：ok，尝试次数：94] - 2 > 67 > 1985
[剩余数量：992] - [当前时间：16:02:03]

[状态：ok，尝试次数：14] - 2 > 67 > 2011
[剩余数量：991] - [当前时间：16:02:03]

[状态：ok，尝试次数：13] - 2 > 67 > 2015
[剩余数量：990] - [当前时间：16:02:03]

[状态：ok，尝试次数：2] - 2 > 67 > 2023
[剩余数量：989] - [当前时间：16:02:03]

[状态：ok，尝试次数：26] - 2 > 67 > 2010
[剩余数量：988] - [当前时间：16:02:04]

[状态：ok，尝试次数：43] - 2 > 67 > 2006
[剩余数量：987] - [当前时间：16:02:04]

[状态：ok，尝试次数：8] - 2 > 68 > 1993
[剩余数量：986] - [当前时间：16:02:05]

[状态：ok，尝试次数：61] - 2 > 67 > 2001
[剩余数量：985] - [当前时间：16:02:05]

[状态：ok，尝试